In [ ]:
#Init packages that I'll need

# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# data
FOLDERNAME = 'cs224u_project/data/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# this ensures that the Python interpreter of the Colab VM can load
# python files from within it.
import sys
import json

sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [ ]:
import pandas as pd
import os

In [ ]:
QUERIES_TEST_FILENAME = os.path.join(sys.path[-1], 'education_test_queries_withqrels.tsv')
QUERIES_TRAIN_FILENAME = os.path.join(sys.path[-1], 'education_train_queries_withqrels.tsv')

QRELS_TRAIN_FILENAME = os.path.join(sys.path[-1], 'qrels_train_pruned.tsv')

queries_test = pd.read_table(QUERIES_TEST_FILENAME,header=None)
queries_train = pd.read_table(QUERIES_TRAIN_FILENAME,header=None)
qrels_train = pd.read_table(QRELS_TRAIN_FILENAME,header=None)




In [ ]:
%%capture
!pip install pyserini==0.12.0

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
from pyserini.search import SimpleSearcher
searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')


Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [01:50, 20.1MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...


In [ ]:
import json

Sparse Retrieval
### The SimpleSearcher class provides the entry point for sparse retrieval using bag-of-words representations. Anserini supports a number of pre-built indexes for common collections that it'll automatically download for you and store in ~/.cache/pyserini/indexes/. Here's how to use a pre-built index for the MS MARCO passage ranking task and issue a query interactively:

In [ ]:

class searchBM25RankingsByQuery():
  def compute_hits(query,k):  
      hits = searcher.search(query,k)
      for i in range(0, k):
        print(f'{i+1:2} {hits[i].docid:7} {hits[i].score:.5f}')

  def inspect_passage_contents(query,num):
    hits = searcher.search(query)
    print(hits[num].raw)


  
    

In [ ]:
ranked_passages = searchBM25RankingsByQuery.compute_hits('how long must one work in education in order to become an administrator?',k=1000)


 1 5556003 15.82610
 2 5712991 14.44710
 3 5893830 13.59730
 4 5269964 13.20740
 5 6471008 12.96260
 6 4734283 12.88790
 7 5269961 12.51650
 8 6936553 12.50230
 9 2454062 12.46220
10 7488438 12.43440
11 2415306 12.41890
12 5785842 12.41410
13 6028822 12.32550
14 5893825 12.32270
15 4290789 12.31760
16 6341804 12.29300
17 4734288 12.05650
18 3067593 11.96910
19 157713  11.89050
20 1987201 11.87060
21 1608560 11.83680
22 2509600 11.82640
23 3960733 11.70150
24 5861996 11.68890
25 3298058 11.67300
26 5756912 11.64060
27 6701623 11.63750
28 2211854 11.63310
29 6629043 11.59040
30 2523622 11.56120
31 352708  11.55530
32 1346036 11.52960
33 7667389 11.50630
34 4907771 11.48210
35 5316520 11.39700
36 1356133 11.39000
37 53121   11.36750
38 6250730 11.35550
39 7451958 11.32440
40 3549330 11.27420
41 6204830 11.24940
42 3288220 11.22800
43 4001804 11.22080
44 5060736 11.22060
45 2175215 11.20680
46 3067594 11.20020
47 5383898 11.17970
48 1126258 11.16800
49 6238452 11.16380
50 6209904 11.16310


In [ ]:
inspect_passage = searchBM25RankingsByQuery.inspect_passage_contents('how long must one work in education in order to become an administrator?',0)


NameError: ignored

In [ ]:
#Another helper function to give us the passage given the passage_id
searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')
doc = searcher.doc('5287878')
json_doc = json.loads(doc.raw())
print(json_doc)


Attempting to initialize pre-built index msmarco-passage.
/root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d already exists, skipping download.
Initializing msmarco-passage...
{'id': '5287878', 'contents': "And the statistic is taken from a website I trust and use at work. As far as I can tell, I have a few options: 1. Leave it as is. 2. Add a citation (Source) at the end of the sentence. 3. Add a citation *source at the end of the essay. 3. Rephrase the statistic (remove quote marks) and don't cite. 4. Rephrase the statistic and cite. 5. Take out the statistic altogether. Obviously I don't want to do anything even vaguely hinting of plagiarism."}


In [ ]:
queries_test.head(1000)

,0,1
0,1008167,which cycle includes bacteria taking gas from ...
1,548475,westborough local history librarian
2,940049,when did newton discover theory of light and c...
3,985122,where is italy located geographically
4,995972,definition of adsorbed in science
...,...,...
895,1100286,george mason women's history month
896,59084,calling powershell functions
897,627416,what documents are required for learners permit
898,117679,define angle geometry


In [ ]:
#for each query, find the relevant passage
def computeTopKRankings(df,k):
  #query_id, passage_id, rank
  queryIdCol = df.iloc[:, 0]
  queryCol = df.iloc[:, 1]
  # print(queryCol[0])
  list_to_fill = []
  for i in range(0,900):
    query = queryCol[i]
    query_id = queryIdCol[i]
    #get the top k BM25 rankings
    hits = searcher.search(str(query),k)

    for j in range(0, k):
      #generate df in form query_id, passage_id, rank
      rank = j+1
      passage_id = hits[j].docid
      entry = [query_id,passage_id,rank]
      list_to_fill.append(entry)
  return pd.DataFrame( list_to_fill )

In [ ]:
#Now, convert this list to a df 
new_dev_df = computeTopKRankings(queries_test,k=100)


In [ ]:
new_dev_df.head(1000000000)

,0,1,2
0,1008167,7737615,1
1,1008167,5126562,2
2,1008167,1908077,3
3,1008167,2452044,4
4,1008167,312391,5
...,...,...,...
899995,1101488,3134948,996
899996,1101488,3282275,997
899997,1101488,364105,998
899998,1101488,364106,999


In [ ]:
#Now, export to table
new_dev_df.to_csv('k100_ranked_education_test_queries_withqrels.tsv', sep = "\t",index=False, header=False)

In [ ]:

qrels_train.columns = ['query_id','ignore',"passage_id", "ignore2"]
queries_train.columns = ['query_id','query']

This part of the notebook, we'll generate a training file
qid pid neg_pid

In [ ]:
#To make indexing simpler, we'll drop duplicates from the qrels.
# qrels_train = qrels_train.drop_duplicates(subset=['0'])
# qrels_train.head(1000000)
queryIdCol = queries_train.iloc[:, 0]
queryCol = queries_train.iloc[:, 1]
queryCol = queries_train.iloc[:, 1]

def computeNegEx():
  ##Find a random passage id
  random_query_id = queryIdCol.sample(n=1).values
  random_query_id = random_query_id[0]
  # print(random_query_id)
  row = qrels_train.loc[qrels_train['query_id'] == random_query_id]
  neg_examples = row['passage_id'].values
  neg_example = neg_examples[0]
  return neg_example

def computePosEx(queryId):
  row = qrels_train.loc[qrels_train['query_id'] == queryId]
  pos_examples = row['passage_id'].values
  pos_example = pos_examples[0]
  return pos_example

#MUST BE A STRING OR BREAKS COMPLETELY!!!!
def findPassage(passage_id):
  doc = searcher.doc(str(passage_id))
  json_doc = json.loads(doc.raw())
  result = json_doc['contents']
  return result


def generate_training_triples():
  # queryCol = qrels_train.iloc[:, 1]
  list_to_fill = []
  length = len(queries_train)
  for i in range(length):
    query = queryCol[i]
    # print(query)
    queryId = queryIdCol[i]
    pos_example_id = computePosEx(queryId)
    pos_example = findPassage(pos_example_id)
    # print(pos_example)
    neg_example_id = computeNegEx()
    neg_example = findPassage(neg_example_id)
    list_to_fill.append([query,pos_example, neg_example])
    # print(neg_example)
  return list_to_fill

list_to_fill = generate_training_triples()


In [ ]:
triples_train = pd.DataFrame( list_to_fill )
triples_train.head(10000000)


,0,1,2
0,do plant cells have an extracellular matrix,Trending Now. Answers. The building technologi...,"American jeans used to mean Levi, Wrangler, an..."
1,how to find a linear relationship of a scatter...,Scatter Plots. A Scatter (XY) Plot has points ...,Types of Distance Learning There are two dista...
2,what is the scope of psychology,Answer: psychology is the scientific study of ...,"In ideal risk management, a prioritization pro..."
3,"in muscle cells, fermentation produces",Lactic Acid Fermentation. Glycolysis is a meta...,Technology for Teaching: 10 Ways to Improve Cl...
4,meaning of chemical change in science,"By Anne Marie Helmenstine, Ph.D. Chemical Chan...","Fiber optics (optical fibers) are long, thin s..."
...,...,...,...
9684,what is an abstract science fair,An abstract is an abbreviated version of your ...,What is the name of the ancient greek who wrot...
9685,what's a null hypothesis,The null hypothesis states that any effects ob...,Leadership is the ability to help people achie...
9686,who plays howard s mom on big bang theory,The actress best known for playing the unseen ...,Professional Education Jefferson's medical edu...
9687,eccentric definition anatomy,"eccentric contraction, a type of muscle contra...",Quick Answer. Glycogen is a polysaccharide tha...


In [ ]:
#Now, export
triples_train.to_csv('education_triples_train.tsv', sep = "\t",index=False, header=False)